In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
import operator
from typing import Annotated, List, Tuple, TypedDict

from langchain_core.pydantic_v1 import BaseModel


class PlanSimulateState(BaseModel):
    # User's original intent
    input: str
    # List of steps (descriptions) generated to be simulated.
    steps: List[str]
    # List of transactions that have been successfully simulated; each tuple contains (description, transaction_params)
    simulated_txs: Annotated[List[Tuple], operator.add]
    # The final response.
    response: str

### Planner Node


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from case.case_retriever import get_retriever
from models.case import BatchCase


system_prompt = """
You are an expert case generator specializing in EVM blockchain transactions. Your task is to convert a description into the simplest possible set of actionable steps.

Rules:
- Ensure the output is concise and limited to the essential action.
- Avoid unnecessary technical details (e.g., creating a transaction, signing, broadcasting) unless explicitly requested by the user.
- Focus on the high-level intent of the task, not the technical implementation.
- If the user's intent is unclear, refine it as needed to generate a coherent set of steps.
- Do not include steps that require user interaction (e.g., asking for confirmations or actions on the blockchain).
- Only generate steps directly relevant to EVM blockchain transactions and in line with the user's high-level request.

Context:
{context}"""


def format_docs(docs) -> str:
    return "\n\n".join(str(doc.metadata) for doc in docs)


retriever = get_retriever()

user_intent = "Description: {description}"

planner_prompt = ChatPromptTemplate.from_messages([system_prompt, user_intent])

planner_model = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)

planner = (
    {"context": retriever | format_docs, "description": RunnablePassthrough()}
    | planner_prompt
    | planner_model.with_structured_output(BatchCase)
)

In [4]:
case = planner.invoke("Send 0.01 ETH to 0x8c575b178927fF9A70804B8b4F7622F7666bB360")
print(case)

ID: send_eth_0.01
Description: Send 0.01 ETH to a specified address on the Ethereum blockchain.
Steps:
1. Send 0.01 ETH to the address 0x8c575b178927fF9A70804B8b4F7622F7666bB360.


In [5]:
case = planner.invoke("Send 100 USDT to scott.")
print(case)

ID: send_usdt_to_scott
Description: Send 100 USDT to Scott.
Steps:
1. Transfer 100 USDT to Scott's Ethereum address.


In [6]:
case = planner.invoke("Swap 100 USDT to USDC")
print(case)

ID: swap_usdt_to_usdc
Description: Swap 100 USDT to USDC
Steps:
1. Approve 100 USDT for swapping on a decentralized exchange.
2. Swap 100 USDT to USDC on the chosen decentralized exchange.


In [7]:
case = planner.invoke("Stake 0.03 ETH with Lido and deposit to Eigenpie")
print(case)

ID: stake_eth_eigenpie
Description: Stake 0.03 ETH with Lido and deposit to Eigenpie
Steps:
1. Stake 0.03 ETH to Lido to receive stETH.
2. Approve stETH to be deposited to Eigenpie.
3. Deposit stETH to Eigenpie.


### Simulation Node
